In [ ]:
!pip install numpy matplotlib networkx

# **GENI**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

nodos_fijos = np.array([
    [np.random.randint(0, 100), np.random.randint(0, 100)] for _ in range(200)
])

def calcular_distancia(nodo1, nodo2):
    return np.linalg.norm(nodo1 - nodo2)

def crear_matriz_distancias(nodos):
    n = len(nodos)
    matriz = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            matriz[i, j] = calcular_distancia(nodos[i], nodos[j])
    return matriz

def geni_insertion(nodos, matriz_distancias):
    recorrido = [0]
    dibujar_ruta(nodos, recorrido)

    while len(recorrido) < len(nodos):
        candidato = min([i for i in range(len(nodos)) if i not in recorrido])
        mejor_pos = None
        mejor_aumento = float('inf')
        for i in range(len(recorrido)):
            j = (i + 1) % len(recorrido)
            aumento = (matriz_distancias[recorrido[i], candidato] +
                       matriz_distancias[candidato, recorrido[j]] -
                       matriz_distancias[recorrido[i], recorrido[j]])
            if aumento < mejor_aumento:
                mejor_aumento = aumento
                mejor_pos = i + 1
        recorrido.insert(mejor_pos, candidato)
        dibujar_ruta(nodos, recorrido)

    return recorrido

def dibujar_ruta(nodos, ruta):
    G = nx.Graph()
    for i in range(len(ruta)):
        G.add_edge(ruta[i], ruta[(i + 1) % len(ruta)])

    pos = {i: nodos[i] for i in range(len(nodos))}

    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=300, font_size=6)
    plt.show()

matriz_distancias = crear_matriz_distancias(nodos_fijos)

ruta = geni_insertion(nodos_fijos, matriz_distancias)


# GENIUS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

nodos_fijos = np.array([
    [np.random.randint(0, 100), np.random.randint(0, 100)] for _ in range(10) #n
])

def calcular_distancia(nodo1, nodo2):
    return np.linalg.norm(nodo1 - nodo2) #euclidiana con norma .linalg.norm dif(x1-x2)^2 sqrt

def crear_matriz_distancias(nodos):
    n = len(nodos)
    matriz = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            matriz[i, j] = calcular_distancia(nodos[i], nodos[j])
    return matriz

def geni_insertion(nodos, matriz_distancias):
    recorrido = [0]
    dibujar_ruta(nodos, recorrido, None, 0)

    iteracion = 1
    nodo_anterior = None

    while len(recorrido) < len(nodos):
        candidato = min([i for i in range(len(nodos)) if i not in recorrido])
        mejor_pos = None
        mejor_aumento = float('inf')
        # Insertar el candidato en la mejor posición
        for i in range(len(recorrido)):
            j = (i + 1) % len(recorrido)
            aumento = (matriz_distancias[recorrido[i], candidato] +
                       matriz_distancias[candidato, recorrido[j]] -
                       matriz_distancias[recorrido[i], recorrido[j]])
            if aumento < mejor_aumento:
                mejor_aumento = aumento
                mejor_pos = i + 1
        recorrido.insert(mejor_pos, candidato)
        dibujar_ruta(nodos, recorrido, candidato, iteracion)  # Mostrar la ruta en cada inserción

        nodo_anterior = candidato
        iteracion += 1

    return recorrido

# US
def us_post_optimization(nodos, matriz_distancias, ruta, p=6):
    iteracion = len(ruta) + 1
    for _ in range(2):  # Iterar n veces para mayor optimización (Vueltas)
        for i in range(1, len(ruta) - 1):
            nodo = ruta.pop(i)  # Eliminar el nodo actual del ciclo
            mejor_pos = None
            mejor_aumento = float('inf')

            # Considerar p vecinos más cercanos
            vecinos = sorted(range(len(nodos)), key=lambda x: matriz_distancias[nodo, x])[:p]

            for j in range(len(ruta)):
                aumento = (matriz_distancias[ruta[j], nodo] +
                           matriz_distancias[nodo, ruta[(j + 1) % len(ruta)]] -
                           matriz_distancias[ruta[j], ruta[(j + 1) % len(ruta)]])
                if aumento < mejor_aumento:
                    mejor_aumento = aumento
                    mejor_pos = j + 1
            ruta.insert(mejor_pos, nodo)  # Reinsertar el nodo
            dibujar_ruta(nodos, ruta, nodo, iteracion)  # ruta actualizada
            iteracion += 1

    return ruta

def dibujar_ruta(nodos, ruta, nodo_resaltado, iteracion):
    G = nx.Graph()
    for i in range(len(ruta)):
        G.add_edge(ruta[i], ruta[(i + 1) % len(ruta)])  # Conectar nodos

    pos = {i: nodos[i] for i in range(len(nodos))}  # Posiciones de los nodos

    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=300, font_size=6)

    # Resaltar el nodo reciente
    if nodo_resaltado is not None:
        nx.draw_networkx_nodes(G, pos, nodelist=[nodo_resaltado], node_color='red', node_size=500)

    plt.title(f"Iteración: {iteracion}")
    plt.show()

matriz_distancias = crear_matriz_distancias(nodos_fijos)

ruta = geni_insertion(nodos_fijos, matriz_distancias)

ruta_optimizada = us_post_optimization(nodos_fijos, matriz_distancias, ruta, p=6) #:=)
